In [4]:
!pip install -e .
!pip install spacy
!python3 -m spacy download en_core_web_lg
!pip install pandas
!pip install openpyxl


Obtaining file:///Users/rajashreedahal/Desktop/Metadata_Finetuning/larch
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done


  Building editable for larch (pyproject.toml) ... done
  Created wheel for larch: filename=larch-0.0.2a0-0.editable-py3-none-any.whl size=14420 sha256=8654cee9105820fd3405570201e026b260b5f055fb18cf879113365ef9beb317
  Stored in directory: /private/var/folders/m4/9_ydsvdn3b3d84rntrrsxm0h0000gn/T/pip-ephem-wheel-cache-bp9ptwer/wheels/3a/98/39/46f8f1efc02842106044493327f82ab1f2c43a0a7f389a1df9
Successfully built larch
  Attempting uninstall: larch
    Found existing installation: larch 0.0.2a0
    Uninstalling larch-0.0.2a0:
      Successfully uninstalled larch-0.0.2a0

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 13.6 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [60]:
import pandas as pd
from larch.metadata import InstructorBasedOpenAIMetadataExtractor
from larch.metadata.validators import WhitelistBasedMetadataValidator
from larch.processors import PIIRemover, TextProcessingPipeline
from larch.utils import load_whitelist

text_processor = TextProcessingPipeline(
    lambda x: re.sub(r"\$(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\)(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\#(?=\w|\n|\()", " ", x).strip(),
    lambda x: x.replace("\t", " ").replace("!", " ").strip(),
    PIIRemover()
)
import re
import requests
from bs4 import BeautifulSoup

from pydantic import BaseModel,constr,Field
from typing import Literal

class Environmental_Justice(BaseModel):
    Indicators: Literal["Disasters","Human Dimensions","Food Availability","Health & Air Quality","Water Availability", "Extreme Heat","Urban Flooding","Climate"]
    Description: str
    Geographic_Coverage: str =Field(pattern=r'')
    Format: str
    Spatial_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$')
    Temporal_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$')
    Temporal_Extent: str = Field(pattern=r'^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$')
    Latency: str = Field(pattern=r'^\d+(\s*(Day|days|months|NRT))?$')
    Project: str
    Data_Visualization: bool

    

    
schema = Environmental_Justice
sheet_names=["Water Availability - Cleaned", "Disasters (Disaster Recovery) -", 
 "Food Availability - Cleaned", "Human Dimensions - Cleaned", "Urban Flooding - Cleaned",
 "Extreme Heat - CIP","Health and Air Quality - CIP", "Climate (Climate Change) - CIP"]

mapper = pd.read_excel("nasa_esds.xlsx", sheet_name=sheet_names)
res = {}
url_lists=[]
for field_key, value_df in mapper.items():
    res[field_key] = {}
    cols = value_df.columns
    for i, row in value_df.iterrows():
        source_link_col = [col for col in cols if col == "Source/Link"]
        alternate_vals = filter(None, row[source_link_col])
        alternate_vals=list(alternate_vals)
        if alternate_vals and str(alternate_vals[0]).startswith("http"):
            url_lists.append(alternate_vals[0])

url_content=[]
for url in url_lists[0:1]:
    url="https://doi.org/10.7927/3xxe-ap97"
    response=requests.get(url)
    html_page = response.text
    soup = BeautifulSoup(html_page, "html.parser")
    text=soup.get_text()
    text = re.sub(r'[\t\n\r\f\v]+', '', text)
    text=re.sub(r'[^\w\s]', '', text)



In [54]:
text

'  Global Gridded Relative Deprivation Index GRDI v1 Poverty Mapping  SEDAC Socioeconomic Data and Applications Center sedacA Data Center in NASAs Earth Observing System Data and Information System EOSDIS  Hosted by CIESIN at Columbia UniversitySearch SEDACDataData SetsMapsWeb Pagessearch Data Data Sets Data Collections Featured Data Uses Data Citations Citations Database Data SubmissionMaps Map Gallery Map Viewer Map Services Mapping ToolsThemes Agriculture Climate Conservation Governance Hazards Health Infrastructure Land Use Marine and Coastal Population Poverty Remote Sensing Sustainability Urban WaterResources Guides Multimedia Networks News Publications Related Sites Remote Sensing ToolsSocial Media Twitter FaceBook YouTube Flickr Blog Posts CommunitiesAbout About SEDAC User Working Group Privacy User RegistrationHelp\xa0Poverty Mapping  Follow Us TwitterFollow Us on FacebookYouTubeFlickr\xa0\xa0\xa0\xa0\xa0\xa0Share  TwitterFacebookCollection OverviewMethodsData Sets 7Global Gri

In [61]:
metadata_extractor = InstructorBasedOpenAIMetadataExtractor(
    model="gpt-3.5-turbo-0125",
#     model="gpt-4",
    schema=schema,
    preprocessor=text_processor,
    debug=True,
)

text = text

metadata = metadata_extractor(text)


2024-02-07 11:44:15.657 | DEBUG    | larch.metadata.extractors_openai:_extract:73 - nchars=3524
Text :: Global Gridded Relative Deprivation Index GRDI v1 Poverty Mapping  SEDAC Socioeconomic Data and Applications Center sedacA Data Center in NASAs Earth Observing System Data and Information System EOSDIS  Hosted by CIESIN at Columbia UniversitySearch SEDACDataData SetsMapsWeb Pagessearch Data Data Sets Data Collections Featured Data Uses Data Citations Citations Database Data SubmissionMaps Map Gallery Map Viewer Map Services Mapping ToolsThemes Agriculture Climate Conservation Governance Hazards Health Infrastructure Land Use Marine and Coastal Population Poverty Remote Sensing Sustainability Urban WaterResources Guides Multimedia Networks News Publications Related Sites Remote Sensing ToolsSocial Media Twitter FaceBook YouTube Flickr Blog Posts CommunitiesAbout About SEDAC User Working Group Privacy User RegistrationHelp Poverty Mapping  Follow Us TwitterFollow Us on FacebookYouTubeF

the system prompt is Extract metadata and entities details accurately from my requests.
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.'}, {'role': 'user', 'content': 'Global Gridded Relative Deprivation Index GRDI v1 Poverty Mapping  SEDAC Socioeconomic Data and Applications Center sedacA Data Center in NASAs Earth Observing System Data and Information System EOSDIS  Hosted by CIESIN at Columbia UniversitySearch SEDACDataData SetsMapsWeb Pagessearch Data Data Sets Data Collections Featured Data Uses Data Citations Citations Database Data SubmissionMaps Map Gallery Map Viewer Map Services Mapping ToolsThemes Agriculture Climate Conservation Governance Hazards Health Infrastructure Land Use Marine and Coastal Population Poverty Remote Sensing Sustainability Urban WaterResources Guides Multimedia Networks News Publications Related Sites Remote Sensing ToolsSocial Media Twitter FaceBook YouTube Flickr Blog Posts Communitie

In [62]:
import json
response=json.loads((metadata['choices'][0]['message']["function_call"]["arguments"]))
print(response)

{'Indicators': 'Poverty', 'Description': 'Global Gridded Relative Deprivation Index (GRDI) v1 is a dataset that characterizes the relative levels of multidimensional deprivation and poverty in each 30 arcsecond (1 km) pixel. It provides a value from 0 to 100, where 100 represents the highest level of deprivation and 0 the lowest.', 'Geographic_Coverage': 'Global', 'Format': 'Raster map', 'Spatial_Resolution': '1 km', 'Temporal_Resolution': 'N/A', 'Temporal_Extent': '2010-2020', 'Latency': 'N/A', 'Project': 'NASA Socioeconomic Data and Applications Center (SEDAC) at Columbia University', 'Data_Visualization': True}
